<a href="https://colab.research.google.com/github/kimdw92/kaggle_study/blob/main/document_clustering_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Opinion Review 데이터 세트를 이용한 문서 군집화(비지도 학습)

UCI Machine Learning Repository에 있는 Opinion Review 데이터 세트 이용

https://archive.ics.uci.edu/ml/datasets/Opinosis+Opinion+%26frasl%3B+Review

In [5]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### filename과 opinion의 2가지 col을 가지는 데이타프레임 생성

In [2]:
import pandas as pd
import glob ,os

path = r'/content/drive/MyDrive/datasets/OpinosisDataset1.0/OpinosisDataset1.0/topics'                     
# path로 지정한 디렉토리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합, 
# 개별 파일들의 파일내용은 DataFrame로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합 
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    
    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경. 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

,filename,opinion_text
0,gas_mileage_toyota_camry_2007,...
1,bathroom_bestwestern_hotel_sfo,...
2,features_windows7,...
3,fonts_amazon_kindle,...
4,free_bestwestern_hotel_sfo,...


### TF-IDF 피쳐 벡터화

In [3]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## KMeans 기반의 군집화 수행

문서는 크게 전자제품, 자동차, 호텔로 주제가 나누어진다.

In [7]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [8]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,gas_mileage_toyota_camry_2007,...,0
1,bathroom_bestwestern_hotel_sfo,...,1
2,features_windows7,...,0
3,fonts_amazon_kindle,...,4
4,free_bestwestern_hotel_sfo,...,1


In [9]:
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,comfort_honda_accord_2008,...,0
17,comfort_toyota_camry_2007,...,0
2,features_windows7,...,0
0,gas_mileage_toyota_camry_2007,...,0
9,interior_honda_accord_2008,...,0
24,interior_toyota_camry_2007,...,0
8,mileage_honda_accord_2008,...,0
23,performance_honda_accord_2008,...,0
13,quality_toyota_camry_2007,...,0
47,seats_honda_accord_2008,...,0


cluster 0은 자동차에 대한 리뷰로 군집화 돼있다.

In [10]:
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
1,bathroom_bestwestern_hotel_sfo,...,1
27,food_holiday_inn_london,...,1
21,food_swissotel_chicago,...,1
4,free_bestwestern_hotel_sfo,...,1
26,location_bestwestern_hotel_sfo,...,1
22,location_holiday_inn_london,...,1
28,parking_bestwestern_hotel_sfo,...,1
30,price_holiday_inn_london,...,1
19,room_holiday_inn_london,...,1
33,rooms_bestwestern_hotel_sfo,...,1


cluster 1은 호텔에 대한 리뷰로 군집화 돼잇다.

중심 개수를 5개에서 3개로 낮춰서 다시 군집화

In [11]:
# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
34,service_holiday_inn_london,...,0
33,rooms_bestwestern_hotel_sfo,...,0
22,location_holiday_inn_london,...,0
41,service_bestwestern_hotel_sfo,...,0
26,location_bestwestern_hotel_sfo,...,0
43,staff_swissotel_chicago,...,0
21,food_swissotel_chicago,...,0
28,parking_bestwestern_hotel_sfo,...,0
27,food_holiday_inn_london,...,0
30,price_holiday_inn_london,...,0


## 군집별 핵심 단어 추출하기

In [12]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00084138 0.         ... 0.         0.         0.        ]
 [0.0104721  0.         0.         ... 0.00735716 0.         0.        ]]


cluster_centers의 값은 0에서 1까지의 값을 가질 수 있고 1에 가까울수록 중심과 가까운 피쳐라는 뜻이다

In [13]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1]
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details

In [14]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')

In [15]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['bathroom_bestwestern_hotel_sfo', 'free_bestwestern_hotel_sfo', 'room_holiday_inn_london', 'food_swissotel_chicago', 'location_holiday_inn_london', 'location_bestwestern_hotel_sfo', 'food_holiday_inn_london']
####### Cluster 1
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'quality', 'gas mileage', 'transmission', 'car', 'performance']
Reviews 파일명 : ['gas_mileage_toyota_camry_2007', 'mileage_honda_accord_2008', 'interior_honda_accord_2008', 'quality_toyota_camry_2007', 'comfort_toyota_camry_2007', 'comfort_honda_accord_2008', 'performance_honda_accord_2008']
####### Cluster 2
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'voice', 'size', 'map']
Reviews 파일명 : ['features_windows7', 'fonts_amazon_kindle', 'accuracy_garmin_nuvi_255W_gps', 'battery-life_ipod_nano_8gb', 'directio